In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset('smilegate-ai/kor_unsmile')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data_train = dataset['train']
data_valid = dataset['valid']

print(data_train[0])

{'문장': '일안하는 시간은 쉬고싶어서 그런게 아닐까', '여성/가족': 0, '남성': 0, '성소수자': 0, '인종/국적': 0, '연령': 0, '지역': 0, '종교': 0, '기타 혐오': 0, '악플/욕설': 0, 'clean': 1, '개인지칭': 0, 'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]}


In [ ]:
from transformers import BertTokenizer,BertForSequenceClassification
import torch

model_name = 'klue/roberta-base'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(data_train['labels'][0]))

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.laye

In [ ]:
train_texts = data_train['문장']
train_labels = data_train['labels']

val_texts = data_valid['문장']
val_labels = data_valid['labels']

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
    self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    self.labels = torch.tensor(labels, dtype=torch.float32)

  def __getitem__(self, idx):
    item = {key: val[idx] for key, val in self.encodings.items()}
    item['labels'] = self.labels[idx]
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length=50)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length=50)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=10)
val_loader = DataLoader(val_dataset, batch_size=10)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05,

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
from sklearn.metrics import label_ranking_average_precision_score

In [ ]:
from tqdm import tqdm

epochs = 2
all_preds = []
all_labels = []


for epoch in tqdm(range(epochs)):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # 검증 및 스코어 표시
    model.eval()
    scores = []

    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss = outputs.loss
            scores.append(label_ranking_average_precision_score(labels.cpu().numpy(), outputs.logits.cpu().numpy()))
            all_preds.extend(torch.sigmoid(outputs.logits).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_score = sum(scores) / len(scores)
    print(f'Epoch {epoch + 1}: Validation Score - {avg_score:.4f}')

    if avg_score > 0.89:
        break

 50%|█████     | 1/2 [03:09<03:09, 189.78s/it]

Epoch 1: Validation Score - 0.8041


100%|██████████| 2/2 [06:05<00:00, 182.68s/it]

Epoch 2: Validation Score - 0.7969


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

all_preds = np.array(all_preds) > 0.5
report = classification_report(np.array(all_labels), all_preds)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.60      0.67       788
           1       0.85      0.66      0.75       668
           2       0.80      0.82      0.81       560
           3       0.70      0.72      0.71       852
           4       0.86      0.70      0.77       292
           5       0.80      0.82      0.81       520
           6       0.84      0.82      0.83       580
           7       0.50      0.04      0.08       268
           8       0.52      0.54      0.53      1572
           9       0.69      0.56      0.61      1870

   micro avg       0.70      0.63      0.66      7970
   macro avg       0.73      0.63      0.66      7970
weighted avg       0.70      0.63      0.65      7970
 samples avg       0.64      0.63      0.63      7970



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
